<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/HousePrices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Authenticate User </h3>

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# importing libraries
import pandas as pd


<h3>FEtching data from clo9ud storage </h3>

In [23]:
!gsutil cp gs://mm_tensorflow_bucket/pp-2018-part2.csv	 /tmp/pp-2018-part2.csv
!gsutil cp gs://mm_tensorflow_bucket/pp-2015-part1.csv	 /tmp/pp-2015-part1.csv
  
  

Copying gs://mm_tensorflow_bucket/pp-2018-part2.csv...
- [1 files][ 84.7 MiB/ 84.7 MiB]                                                
Operation completed over 1 objects/84.7 MiB.                                     
Copying gs://mm_tensorflow_bucket/pp-2015-part1.csv...
- [1 files][ 83.4 MiB/ 83.4 MiB]                                                
Operation completed over 1 objects/83.4 MiB.                                     


<h3> Readign the dataset from Land Registry</h3>

In [0]:
def read_land_registry_datasets(filename):
  
  df =  pd.read_csv(filename, names=['unique_id',	'price_paid','deed_date','postcode',
                                                       'property_type',	'new_build',	'estate_type',	
                                                       'saon',	'paon',	'street','locality',
                                                       'town',	'district',	'county',	'transaction_category','linked_data_uri']).fillna('') 
  print ('Found {} in {}'.format(df.shape[0], filename))
  return df

In [0]:
from datetime import datetime
def find_sales_for_postcode(postcode, filenames):
  print('Finding sold prices for postcode:{}'.format(postcode))
  lr_datasets = [read_land_registry_datasets(filename) for filename in filenames]
  
  results = []
  for df in lr_datasets:
    restricted = df[df['postcode'].str.contains('BN2 5')][['price_paid', 'deed_date', 'postcode', 'property_type', 'saon', 'paon', 'street']]
    flats_only = restricted[restricted['property_type'] == 'F']
    print('found {} flats for postcode {} '.format(flats_only.shape[0], postcode))
    results.append(flats_only)
    
  all_res =  pd.concat(results)
  print('All res has shape{}'.format(all_res.shape))
  return all_res

                                        

In [36]:
filenames = ['/tmp/pp-2018-part2.csv', '/tmp/pp-2015-part1.csv']
final = find_sales_for_postcode('BN2 5', filenames)
print('Exportign to csv...with {}'.format(final.shape))
filename = '/tmp/Results.csv'.format(datetime.now().strftime('%Y%m%d-%H:%M'))
final.to_csv(filename, header=True)


Finding sold prices for postcode:BN2 5
Found 511575 in /tmp/pp-2018-part2.csv
Found 504743 in /tmp/pp-2015-part1.csv
found 32 flats for postcode BN2 5 
found 50 flats for postcode BN2 5 
All res has shape(82, 7)
Exportign to csv...with (82, 7)


In [37]:

print('Copying to bucket')
!gsutil cp /tmp/Results.csv gs://mm_tensorflow_bucket/                                       


Copying to bucket
Copying file:///tmp/Results.csv [Content-Type=text/csv]...
/ [1 files][  5.7 KiB/  5.7 KiB]                                                
Operation completed over 1 objects/5.7 KiB.                                      
